<a href="https://colab.research.google.com/github/sihyeon3523/Six_of_cells/blob/modeling/Seunguk_(3)_lyricModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<br>

# 0. Install & import libraries 

In [1]:
from IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons

clear_output()

import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

import sentencepiece

import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm
import ast  # str list to list

1.0.2
2.8.0
4.8.2
0.17.0


In [2]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [3]:
#KoBERT 불러오기
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
clear_output() # clear the output

#토크나이저 생성
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# Import Google Drive 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
os.getcwd()

'/content'

In [6]:
# 경로만 바꾸면 됨!
path = '/content/drive/MyDrive/likelion/finalproj_seunguk'
data_path = 'colab_data/temp_data/'
checkpoint_path = 'colab_data/temp_data/saved_models/' 

os.chdir(path)

# 1. Load the Data

In [7]:
df_allsongs = pd.read_csv('df_allsongs.csv', lineterminator='\n')

In [8]:
df_allsongs.head(3)

,Unnamed: 0,Title,Artist,Date,Genre,Lyric,Lyric_ver2,preprocess_Lyric_ver2,preprocess_Lyric_ver3
0,0,나의 X에게,경서,2022.04.24,발라드,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤...,"['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', ..."
1,1,취중고백,김민석 (멜로망스),2021.12.19,발라드,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...,"['뭐하고 있었니 늦었지만', '잠시 나올래', '너의 집 골목에 있는', '놀이터..."
2,2,사랑은 늘 도망가,임영웅,2021.10.11,"발라드, 국내드라마",눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,"['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지..."


In [9]:
df_allsongs.head(3)

,Unnamed: 0,Title,Artist,Date,Genre,Lyric,Lyric_ver2,preprocess_Lyric_ver2,preprocess_Lyric_ver3
0,0,나의 X에게,경서,2022.04.24,발라드,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤...,"['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', ..."
1,1,취중고백,김민석 (멜로망스),2021.12.19,발라드,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...,"['뭐하고 있었니 늦었지만', '잠시 나올래', '너의 집 골목에 있는', '놀이터..."
2,2,사랑은 늘 도망가,임영웅,2021.10.11,"발라드, 국내드라마",눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,"['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지..."


In [10]:
df_allsongs = df_allsongs[["Title", "Artist", "Date", "Genre", "preprocess_Lyric_ver2", "preprocess_Lyric_ver3"]]
df_allsongs

,Title,Artist,Date,Genre,preprocess_Lyric_ver2,preprocess_Lyric_ver3
0,나의 X에게,경서,2022.04.24,발라드,우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤...,"['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', ..."
1,취중고백,김민석 (멜로망스),2021.12.19,발라드,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...,"['뭐하고 있었니 늦었지만', '잠시 나올래', '너의 집 골목에 있는', '놀이터..."
2,사랑은 늘 도망가,임영웅,2021.10.11,"발라드, 국내드라마",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,"['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지..."
3,듣고 싶을까,MSG워너비(M.O.M),2022.02.26,발라드,우리 함께 듣던 그 노랠 듣고 싶을까 듣고 나서 잠시 날 생각은 할까 아주 혹시라도...,"['우리 함께 듣던 그 노랠 듣고 싶을까', '듣고 나서 잠시 날 생각은 할까', ..."
4,다정히 내 이름을 부르면,경서예지,2021.05.19,발라드,끝없이 별빛이 내리던 밤 기분 좋은 바람이 두 빰을 스치고 새벽 바다 한곳을 보는 ...,"['끝없이 별빛이 내리던 밤', '기분 좋은 바람이', '두 빰을 스치고', '새벽..."
...,...,...,...,...,...,...
3604,애수,진시몬,2004.10.01,성인가요/트로트,아직도 모르겠어 난 정말 꿈이라 생각해야 하는지 너 떠난 그 길목에서 널 ...,"['아직도 모르겠어 난 정말 ', '', '꿈이라 생각해야 하는지 ', '', '너..."
3605,단장의 미아리 고개,송가인,2020.12.26,성인가요/트로트,미아리 눈물고개 임이 넘던 이별 고개 화약 연기 앞을 가려 눈 못 뜨고 헤매일 때 ...,"['미아리 눈물고개', '임이 넘던 이별 고개', '화약 연기 앞을 가려', '눈 ..."
3606,어머님 사랑합니다,송가인,2021.05.27,성인가요/트로트,꽃다운 열아홉에 연지 찍고 시집와서 손발이 터지도록 고생하신 어머님 백날을 하루같이...,"['꽃다운 열아홉에', '연지 찍고 시집와서', '손발이 터지도록', '고생하신 어..."
3607,영동 부르스,송가인,2021.06.13,성인가요/트로트,헤어지기 싫어서 애태우던 그 날 밤 피눈물에 얼룩진 그대의 모습 어차피 떠나야 할 ...,"['헤어지기 싫어서 애태우던 그 날 밤', '피눈물에 얼룩진 그대의 모습', '어차..."


Load saved Model


In [11]:
# 1) Load the Model-builder (function)
with open(data_path + 'model_BERTfunction_v1.pkl', 'rb') as f:
    create_model = dill.loads(pickle.load(f)) # use dill to pickle a python function

# 2) Load the Bert-tokenizer 
with open(data_path + 'tokenizer-bert.pkl', 'rb') as f:
    tokenizer = pickle.load(f) 


# 3) Create the model & load the Model-weights (from checkpoint file)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=300) 

model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


# Classifier as a function

In [15]:
def predict_sentiment(sentence, tokenizer, model):
    
    SEQ_LEN = 300 

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence), # 특수문자 제거
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) 
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    
    new_inputs = (token_ids, token_masks, token_segments)

    # Prediction
    prediction = model.predict(new_inputs) 
    all_predicted_probability = np.round(prediction * 100, 2)
    top_predicted_probability = np.round(np.max(prediction) * 100, 2) 
    
    predicted_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(prediction, axis=1)[0]] # 수정하면 4개 클래스 다 나올듯! 

    print("{}% 확률로 {} 텍스트입니다.".format(top_predicted_probability, predicted_class))
    print(all_predicted_probability)
    return predicted_class, top_predicted_probability, all_predicted_probability # 태깅용 

In [32]:
data = [0.1, 0.3, 0.4, 0.2] 
x = np.array(data)
y = ['분노혐오','놀람공포','슬픔','행복']

In [41]:
for i in range(len(x)):
    print(x[i], y[i])

0.1 분노혐오
0.3 놀람공포
0.4 슬픔
0.2 행복


#### TEST temporary

In [26]:
temp = df_allsongs.preprocess_Lyric_ver2.iloc[0]
temp

'우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤이 새도록 네게 말할 거야 너를 좋아한다고 안녕 그대야 요즘 어떻게 지내 가로수길 걷다가 생각이 나서 행복해 보이는 사람들 속에 우리 둘만 없어서 어색했었어 늘 집에 갈 땐 항상 뒤돌아보면 날 보며 웃어주던 네가 생각났어 우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤이 새도록 네게 말할 거야 너를 좋아한다고 토요일마다 가던 한강공원에 이어폰 하나씩 나눠끼고서 벚꽃엔딩같이 듣고 있으면 부러울 게 없었어 너만 있으면 널 사랑할 때 내가 너무 그리워 날 보며 웃어주던 네가 보고 싶어 우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤이 새도록 네게 말할 거야 너를 좋아한다고 같이 별 보러 갈래 널 다시 만나면 네 옆에 기대서 잠들고 싶어 너의 품에 안길래 이 밤이 새도록 내 곁에 있어줘 내겐 너뿐이라고 '

In [27]:
predict_sentiment(temp, tokenizer, model)

59.05% 확률로 행복 텍스트입니다.


('행복', 59.05, array([[ 4.03,  1.92, 35.01, 59.05]], dtype=float32))

preprocess_Lyric_ver3에 적용 <br> 

list가 str 형태로 row에 저장되었기 때문에 ast library로 다시 list로 바꿔주어야한다.<br>
(str list to list)

In [18]:
temp_list = df_allsongs.preprocess_Lyric_ver2.iloc[0]

In [19]:
temp_list = df_allsongs.preprocess_Lyric_ver3.iloc[0]
print(type(temp_list))
temp_list

<class 'str'>


"['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '안녕 그대야', '요즘 어떻게 지내', '가로수길 걷다가', '생각이 나서', '행복해 보이는 사람들 속에', '우리 둘만 없어서', '어색했었어', '늘 집에 갈 땐', '항상 뒤돌아보면', '날 보며 웃어주던', '네가 생각났어', '우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '토요일마다 가던', '한강공원에', '이어폰 하나씩', '나눠끼고서', '벚꽃엔딩같이 듣고 있으면', '부러울 게 없었어', '너만 있으면', '널 사랑할 때', '내가 너무 그리워', '날 보며 웃어주던', '네가 보고 싶어', '우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '같이 별 보러 갈래', '널 다시 만나면', '네 옆에 기대서', '잠들고 싶어', '너의 품에 안길래', '이 밤이 새도록', '내 곁에 있어줘', '내겐 너뿐이라고', '']"

In [20]:
import ast  

temp_list_2 = ast.literal_eval(temp_list) 
print(type(temp_list_2)) 
print(temp_list_2)

<class 'list'>
['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '안녕 그대야', '요즘 어떻게 지내', '가로수길 걷다가', '생각이 나서', '행복해 보이는 사람들 속에', '우리 둘만 없어서', '어색했었어', '늘 집에 갈 땐', '항상 뒤돌아보면', '날 보며 웃어주던', '네가 생각났어', '우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '토요일마다 가던', '한강공원에', '이어폰 하나씩', '나눠끼고서', '벚꽃엔딩같이 듣고 있으면', '부러울 게 없었어', '너만 있으면', '널 사랑할 때', '내가 너무 그리워', '날 보며 웃어주던', '네가 보고 싶어', '우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '같이 별 보러 갈래', '널 다시 만나면', '네 옆에 기대서', '잠들고 싶어', '너의 품에 안길래', '이 밤이 새도록', '내 곁에 있어줘', '내겐 너뿐이라고', '']


In [21]:
print(temp_list_2[1])
predict_sentiment(temp_list_2[1], tokenizer, model)

예쁘게 빛나던
98.72% 확률로 행복 텍스트입니다.


('행복', 98.72, array([[ 0.4 ,  0.37,  0.5 , 98.72]], dtype=float32))

In [22]:
len(temp_list_2)

55

In [45]:
del temp_list_2[-1]
temp_list_2

['우리 다시 만날래',
 '예쁘게 빛나던',
 '모든 추억들이',
 '너무 그리워',
 '너의 품에 안길래',
 '이 밤이 새도록',
 '네게 말할 거야',
 '너를 좋아한다고',
 '안녕 그대야',
 '요즘 어떻게 지내',
 '가로수길 걷다가',
 '생각이 나서',
 '행복해 보이는 사람들 속에',
 '우리 둘만 없어서',
 '어색했었어',
 '늘 집에 갈 땐',
 '항상 뒤돌아보면',
 '날 보며 웃어주던',
 '네가 생각났어',
 '우리 다시 만날래',
 '예쁘게 빛나던',
 '모든 추억들이',
 '너무 그리워',
 '너의 품에 안길래',
 '이 밤이 새도록',
 '네게 말할 거야',
 '너를 좋아한다고',
 '토요일마다 가던',
 '한강공원에',
 '이어폰 하나씩',
 '나눠끼고서',
 '벚꽃엔딩같이 듣고 있으면',
 '부러울 게 없었어',
 '너만 있으면',
 '널 사랑할 때',
 '내가 너무 그리워',
 '날 보며 웃어주던',
 '네가 보고 싶어',
 '우리 다시 만날래',
 '예쁘게 빛나던',
 '모든 추억들이',
 '너무 그리워',
 '너의 품에 안길래',
 '이 밤이 새도록',
 '네게 말할 거야',
 '너를 좋아한다고',
 '같이 별 보러 갈래',
 '널 다시 만나면',
 '네 옆에 기대서',
 '잠들고 싶어',
 '너의 품에 안길래',
 '이 밤이 새도록',
 '내 곁에 있어줘']

In [46]:
for i in range(len(df_allsongs.preprocess_Lyric_ver3.iloc[0])): 
    temp_list = df_allsongs.preprocess_Lyric_ver3.iloc[0] 
    temp_list = ast.literal_eval(temp_list) 
    del temp_list[-1] # 마지막 원소가 빈칸임
    predict_sentiment(temp_list[i], tokenizer, model)
    print(i)

56.57% 확률로 슬픔 텍스트입니다.
0
98.72% 확률로 행복 텍스트입니다.
1
98.35% 확률로 행복 텍스트입니다.
2
78.42% 확률로 슬픔 텍스트입니다.
3
69.17% 확률로 슬픔 텍스트입니다.
4
70.46% 확률로 슬픔 텍스트입니다.
5
62.93% 확률로 분노혐오 텍스트입니다.
6
36.76% 확률로 놀람공포 텍스트입니다.
7
92.89% 확률로 행복 텍스트입니다.
8
63.51% 확률로 분노혐오 텍스트입니다.
9
80.76% 확률로 놀람공포 텍스트입니다.
10
61.7% 확률로 슬픔 텍스트입니다.
11
97.6% 확률로 행복 텍스트입니다.
12
92.56% 확률로 슬픔 텍스트입니다.
13
83.51% 확률로 놀람공포 텍스트입니다.
14
56.54% 확률로 슬픔 텍스트입니다.
15
76.13% 확률로 슬픔 텍스트입니다.
16
70.67% 확률로 행복 텍스트입니다.
17
60.85% 확률로 놀람공포 텍스트입니다.
18
56.57% 확률로 슬픔 텍스트입니다.
19
98.72% 확률로 행복 텍스트입니다.
20
98.35% 확률로 행복 텍스트입니다.
21
78.42% 확률로 슬픔 텍스트입니다.
22
69.17% 확률로 슬픔 텍스트입니다.
23
70.46% 확률로 슬픔 텍스트입니다.
24
62.93% 확률로 분노혐오 텍스트입니다.
25
36.76% 확률로 놀람공포 텍스트입니다.
26
92.09% 확률로 행복 텍스트입니다.
27
48.61% 확률로 행복 텍스트입니다.
28
85.36% 확률로 놀람공포 텍스트입니다.
29
61.65% 확률로 놀람공포 텍스트입니다.
30
87.22% 확률로 슬픔 텍스트입니다.
31
90.16% 확률로 슬픔 텍스트입니다.
32
44.02% 확률로 놀람공포 텍스트입니다.
33
54.81% 확률로 행복 텍스트입니다.
34
83.71% 확률로 슬픔 텍스트입니다.
35
70.67% 확률로 행복 텍스트입니다.
36
53.64% 확률로 슬픔 텍스트입니다.
37
56.57% 확률로 슬픔 텍스트입니다.
38
98.72% 확률로 행복 텍

IndexError: ignored

# Apply model

In [ ]:
for row in tqdm(df_allsongs.iterrows()):
  # #장르 
  # print(row[1][1])
  # #아티스트 
  # print(row[1][2])
  # #타이틀 
  # print(row[1][3])
  # #가사 
  print(row[1][-1])
  # #발매일 
  # print(row[1][5])
  # print(row[1][-1])
  #print(row)
  break
  #predict_sentiment(row[1][-1], tokenizer, model)

In [ ]:
# Emotion = []
# Probability = []

# for row in tqdm(df_allsongs.iterrows()):
#   predicted_class, predicted_probability = predict_sentiment(row[1][-1], tokenizer, model)
#   #break
#   Emotion.append(predicted_class)
#   Probability.append(predicted_probability)

In [ ]:
# df_allsongs['Emotion'] = Emotion
# df_allsongs['Proba'] = Probability

In [ ]:
# df_allsongs['Emotion'].value_counts()

In [ ]:
# df_allsongs

In [ ]:
# df_allsongs.to_csv('songtagged_1.csv')